In [1]:
# downloading and parsing the initial text file 
import keras 
import numpy as np 

path = keras.utils.get_file('nietzsche.txt',
                           origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Using TensorFlow backend.


Corpus length: 600893


In [ ]:
# vectorizing sequences of characters 
maxlen = 60 
step = 3 
setences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    setences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(setences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(setences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(setences), len(chars)), dtype=np.bool)
for i, setence in enumerate(setences):
    for t, char in enumerate(setences):
        for t, char in enumerate(setence):
            x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [0]:
print(x.shape)
print(y.shape)

In [0]:
# single-layer lstm model for next-character prediction
from keras import layers 

model = keras.models.Sequential() 
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [0]:
# model compilation configuration 
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categoical_crossentropy', optimizer=optimizer)

In [0]:
# function to sample the next character given the model's predictions 
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature 
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    

In [0]:
# text-generation loop 
import random 
import sys 

for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen -1)
    generate_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('-------- temperature: ', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char 
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)                        